In [1]:
from torchvision.models.resnet import resnet50
import torch
import torch.nn as nn
from training.training import Trainer
import os.path as osp
import cv2
import numpy as np

In [2]:
import torch.nn.functional as F

In [3]:
from albumentations import (
    VerticalFlip,
    HorizontalFlip,
    Compose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion,
    OpticalDistortion,
    OneOf,
    CLAHE,
    RandomContrast,
    RandomGamma,
    RandomBrightness,
    Resize)
torch.manual_seed(42)
np.random.seed(42)

2018-11-15 08:58:45,091 [MainThread  ] [DEBUG]  $HOME=/root
2018-11-15 08:58:45,093 [MainThread  ] [DEBUG]  matplotlib data path /opt/conda/lib/python3.6/site-packages/matplotlib/mpl-data
2018-11-15 08:58:45,100 [MainThread  ] [DEBUG]  loaded rc file /opt/conda/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
2018-11-15 08:58:45,102 [MainThread  ] [DEBUG]  matplotlib version 2.2.2
2018-11-15 08:58:45,103 [MainThread  ] [DEBUG]  interactive is False
2018-11-15 08:58:45,105 [MainThread  ] [DEBUG]  platform is linux
2018-11-15 08:58:45,106 [MainThread  ] [DEBUG]  loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.ascii', '_signal', '__main__', 'encodings.utf_8', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'gener

2018-11-15 08:58:45,126 [MainThread  ] [DEBUG]  CACHEDIR=/root/.cache/matplotlib
2018-11-15 08:58:45,129 [MainThread  ] [DEBUG]  Using fontManager instance from /root/.cache/matplotlib/fontList.json


In [4]:
# from fastai.conv_learner import *
# from fastai.dataset import *

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import scipy.optimize as opt

In [5]:
name_label_dict = {
0:  'Nucleoplasm',
1:  'Nuclear membrane',
2:  'Nucleoli',   
3:  'Nucleoli fibrillar center',
4:  'Nuclear speckles',
5:  'Nuclear bodies',
6:  'Endoplasmic reticulum',   
7:  'Golgi apparatus',
8:  'Peroxisomes',
9:  'Endosomes',
10:  'Lysosomes',
11:  'Intermediate filaments',
12:  'Actin filaments',
13:  'Focal adhesion sites',   
14:  'Microtubules',
15:  'Microtubule ends',  
16:  'Cytokinetic bridge',   
17:  'Mitotic spindle',
18:  'Microtubule organizing center',  
19:  'Centrosome',
20:  'Lipid droplets',
21:  'Plasma membrane',   
22:  'Cell junctions', 
23:  'Mitochondria',
24:  'Aggresome',
25:  'Cytosol',
26:  'Cytoplasmic bodies',   
27:  'Rods & rings' }

In [7]:
PATH = './'
TRAIN = '/root/data/protein/train/'
TEST = '/root/data/protein/test/'
LABELS = '/root/data/protein/train.csv'
SAMPLE = '/root/data/protein/sample_submission.csv'


In [8]:
train_names = list({f[:36] for f in os.listdir(TRAIN)})
test_names = list({f[:36] for f in os.listdir(TEST)})
tr_n, val_n = train_test_split(train_names, test_size=0.1, random_state=42)


In [9]:
def open_rgby(path,id): #a function that reads RGBY image
    colors = ['red','green','blue','yellow']
    flags = cv2.IMREAD_GRAYSCALE
    img = [cv2.imread(os.path.join(path, id+'_'+color+'.png'), flags).astype(np.float32)/255
           for color in colors]
    return np.stack(img, axis=-1)

In [10]:
TARGET_SIZE=512

In [11]:
aug = Compose([
    HorizontalFlip(p=0.7),
    RandomGamma(p=0.7),
    #GridDistortion(p=0.6),
    #OpticalDistortion(p=0.6),
    #ElasticTransform(p=0.6),
    Resize(height=TARGET_SIZE, width=TARGET_SIZE)
])

val_aug=Resize(height=TARGET_SIZE, width=TARGET_SIZE)

In [12]:
class ProteinDataset:
    def __init__(self, names, path,aug=aug):
        self.names=names
        self.aug=aug
        self.path=path
        self.labels = pd.read_csv(LABELS).set_index('Id')
        self.labels['Target'] = [[int(i) for i in s.split()] for s in self.labels['Target']]
        
    def __len__(self):
        return len(self.names)
    
    def __getitem__(self, idx):
        
        if(self.path == TEST): label= np.zeros(len(name_label_dict),dtype=np.int)
        else:
            labels = self.labels.loc[self.names[idx]]['Target']
            label=np.eye(len(name_label_dict),dtype=np.float)[labels].sum(axis=0)
        
        img = open_rgby(self.path, self.names[idx])
        img = aug(image=img)['image']
        
        return torch.from_numpy(
            img
        ).permute([2,0,1]), torch.from_numpy(label).float()

In [13]:
train_names, val_names = train_test_split(train_names)

In [14]:
model = resnet50(pretrained=True)

In [15]:
model.conv1 = nn.Conv2d(4,64,(7,7),(2,2),(3,3),bias=False)

In [16]:
model.fc = nn.Linear(model.fc.in_features, 28) # since we have 28 outputs
model.load_state_dict(torch.load('resnet50_best_old.pth.tar'))

model.avgpool = nn.Sequential(
    nn.MaxPool2d(kernel_size=6, stride=2,padding=0),
    nn.AvgPool2d(kernel_size=5, stride=2,padding=0)
)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.Dropout(0.5),
    nn.Linear(512, 28)
)


In [17]:
model.load_state_dict(torch.load('resnet50_best.pth.tar'))

In [22]:
MODEL_NAME='resnet50'
BATCH_SIZE=16
DEVICE=2
EPOCHS=200

### train stage

In [18]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))

        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()

        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        
        return loss.sum(dim=1).mean()


In [19]:
THRESHOLD=0.0

In [20]:
loss = FocalLoss()

In [21]:
def mymetric(pred, target):
    preds = (pred > THRESHOLD).int()
    targs = target.int()
    return (preds==targs).float().mean()

def myloss(pred, target):
    return loss(pred, target)



In [22]:
train_ds = ProteinDataset(train_names, TRAIN)
val_ds = ProteinDataset(val_names, TRAIN, val_aug)

In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
trainer = Trainer(myloss, mymetric, optimizer, MODEL_NAME, None, DEVICE)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_ds,batch_size=BATCH_SIZE)


2018-11-12 06:16:33,809 [Visdom-Socke] [INFO ]  Visdom successfully connected to server


In [24]:
trainer.output_watcher = None

In [25]:
ct = 0
for child in model.children():
    ct += 1
    if ct < 7:
        for param in child.parameters():
            param.requires_grad = False

In [56]:
model.conv1.weight.shape

torch.Size([64, 4, 7, 7])

In [63]:
torch.mean(model.conv1.weight,dim=1).shape

torch.Size([64, 7, 7])

In [64]:
torch.mean(model.conv1.weight,dim=1).unsqueeze(1).shape

torch.Size([64, 1, 7, 7])

In [70]:
w = model.conv1.weight

In [71]:
w.shape

torch.Size([64, 4, 7, 7])

Parameter containing:
tensor([[[[ 3.8840e-02,  1.4477e-01,  8.1565e-02,  ..., -8.3733e-02,
           -5.0097e-02, -9.8249e-02],
          [-5.0880e-02,  1.8462e-02, -1.0346e-02,  ..., -3.6859e-02,
           -1.1928e-01, -7.1972e-02],
          [-1.7249e-01,  6.2043e-02, -3.6353e-02,  ..., -5.0005e-02,
           -8.3304e-02,  3.0776e-02],
          ...,
          [-1.7829e-02, -5.5378e-03,  5.7718e-02,  ..., -7.3080e-03,
           -4.5801e-02, -7.6981e-02],
          [ 7.4269e-02,  2.7448e-02,  2.6845e-02,  ...,  2.1313e-02,
           -7.8128e-02,  3.0925e-02],
          [ 1.0649e-02,  1.5577e-02, -1.8829e-02,  ...,  5.4947e-02,
            3.3654e-02,  9.3044e-02]],

         [[-8.3563e-02,  9.4394e-02,  2.3049e-02,  ..., -1.1455e-01,
           -6.8636e-02, -1.2724e-01],
          [-8.5396e-02,  2.5232e-02, -9.5578e-02,  ...,  1.2756e-01,
           -4.5774e-02,  3.9743e-02],
          [-4.9588e-02,  1.0899e-01, -1.4501e-01,  ...,  1.6635e-01,
            1.3625e-01,  2.0005e-01]

In [26]:
model.to(DEVICE)

ResNet(
  (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [27]:

for i in range(1):
    trainer.train(train_loader, model, i)
    trainer.validate(val_loader, model)

train loss:1.3131424188613892, train metric: tensor(0.9375, device='cuda:0'): : 236it [11:56,  3.04s/it]

KeyboardInterrupt: 

In [ ]:

for child in model.children():
    for param in child.parameters():
        param.requires_grad = True

In [ ]:

for i in range(1):
    trainer.train(train_loader, model, i)
    trainer.validate(val_loader, model)

### inference stage

In [20]:
from tqdm import *

In [23]:
model.to(DEVICE)

ResNet(
  (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [24]:
subm = pd.read_csv(SAMPLE)
test_ds = ProteinDataset(subm.Id.values, TEST, val_aug)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=16)

In [25]:
result = []
for batch_idx, (input, target) in tqdm(enumerate(test_loader)):
    input = input.to(DEVICE)
    result.append(model(input).detach().cpu())
    

732it [11:19,  1.08it/s]


In [26]:
preds = []
for r in result:
    for t in r:
        preds.append(torch.sigmoid(t).numpy())

In [43]:
THRESHOLD=0.3

In [44]:
for i in tqdm_notebook(range(subm.Id.size)):
    subm.iloc[i,1] = ' '.join(np.where(preds[i] > THRESHOLD)[0].astype(str))

In [45]:
np.array([len(k) for k in subm.Predicted.str.split()]).mean()

2.742693556656982

In [46]:
subm.to_csv('my_subm.csv',index=False)